In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By

In [2]:
driver = webdriver.Chrome(executable_path=r"D://chromedriver.exe")

In [43]:
# add your analysis url here
url = "https://tria.ge/231015-b71n6see63/behavioral1/analog?main_event=Files"

In [44]:
driver.get(url)

In [119]:
# Take a pause here until page loads completely

In [59]:
try:
    pagination = driver.find_element(By.CLASS_NAME, "pagination")
    totalClicks = pagination.text.split(" ")[-2]
    totalClicks = int(totalClicks)
except:
    totalClicks = 1
totalClicks


48

In [60]:
try:
    nextButton = pagination.find_elements(By.TAG_NAME, "a")
    # get all class name of next button
    nextButton
    nextbuttonlink = [i.get_attribute("href") for i in nextButton]
except:
    nextbuttonlink = []
nextbuttonlink

['https://tria.ge/231015-b71n6see63/behavioral1/analog?main_event=Files',
 'https://tria.ge/231015-b71n6see63/behavioral1/analog?main_event=Files&offset=8',
 'https://tria.ge/231015-b71n6see63/behavioral1/analog?main_event=Files&offset=10',
 'https://tria.ge/231015-b71n6see63/behavioral1/analog?main_event=Files&offset=47']

In [77]:
import re
pattern = r'offset=[0-9]+$'
text = "dsdsaoffset=10dsds"
match = re.search(pattern, nextButton[-2].get_attribute("href"))
if match:
    found = match.group()
    print(found ,"Pattern found in the URL.")
else:
    print("Pattern not found in the URL.")

offset=47 Pattern found in the URL.


In [61]:
nextbuttonlink[-2][-8:-2]

'ffset='

In [50]:
from collections import defaultdict
data = defaultdict(list)

In [84]:
element = driver.find_element(By.CLASS_NAME, "list").find_elements(By.TAG_NAME, "li")
len(element)

1

In [85]:
temp = defaultdict(list)
for ele in element:
    event = ele.find_element(By.TAG_NAME, "b").text
    # print(event)
    detail = ele.find_elements_by_css_selector(".key-value.rows > div")
    meta = {}
    for i in detail:
        content = i.find_elements(By.TAG_NAME, "div")
        meta[content[0].text] = content[1].text
    temp[event].append(meta)

temp
        

defaultdict(list,
            {'Analog Header': [{'version': '0.2',
               'sample': '231015-b71n6see63',
               'task': '231015-b71n6see63-behavioral1',
               'backend': 'horse2',
               'resource': 'win7-20230831-en',
               'platform': 'windows7_x64'}]})

In [ ]:
import re
pattern = r'offset=[0-9]+$'
for i in range(totalClicks):
    # find elements by their class name
    elements = driver.find_element(By.CLASS_NAME, "list").find_elements(By.TAG_NAME, "li")

    for ele in elements:
        event = ele.find_element(By.TAG_NAME, "b").text
        # print(event)
        detail = ele.find_elements_by_css_selector(".key-value.rows > div")
        meta = {}
        for i in detail:
            content = i.find_elements(By.TAG_NAME, "div")
            meta[content[0].text] = content[1].text
        data[event].append(meta)

    # try to click the "Next" button and go to the next page
    try:
        pagination = driver.find_element(By.CLASS_NAME, "pagination")
        nextButton = pagination.find_elements(By.TAG_NAME, "a")  # replace 'next' with the actual ID or (By.CLASS_NAME, 'next') with the actual class name of the "Next" button
        match = re.search(pattern, nextButton[-2].get_attribute("href"))
        if match:
            found = match.group()
            nextButton[-2].click()
            print("Navigating to Next Page ", found)
        else:
            print("Pattern not found in the URL.")
            break
    except Exception as e:
        print(e)
        print("Reached the end of pages.")
        break
print("Finished scraping.")

In [130]:
main_event = ["Misc","Process","Registry","Files","Mutex","Network"]

In [131]:
id = "231015-b71n6see63"
log_file = f"{id}/log.txt"
with open(log_file, "w") as f:
    f.write(f"Process , 10\n")
    f.write(f"Registry , 13")

In [132]:
try:
    with open(log_file, 'r') as log_file:
        log = log_file.readlines()
        print(log)
        log = log[-1].strip().split(",")
        event = log[0].strip()
        offset = int(log[1].strip())
except:
    event = ""
    offset = 0
event, offset

['Process , 10\n', 'Registry , 13']


('Registry', 13)

In [133]:
main_event

['Misc', 'Process', 'Registry', 'Files', 'Mutex', 'Network']

In [134]:
print(event)
if event in main_event:
    print(main_event.index(event))
else:
    print(-1)

Registry
2


In [143]:
import time
from collections import defaultdict
import json
import re
import os
pattern = r'offset=[0-9]+$'
id = "231015-b71n6see63"

if not os.path.exists(id):
    os.makedirs(id)

log_path = f"{id}/log.txt"
with open(log_path, "r") as f:
    log = f.readlines()
    log = log[-1].strip().split(",")
    logEvent = log[0].strip()
    logOffset = int(log[1].strip())

if logEvent in main_event:
    index = main_event.index(logEvent)
    main_event = main_event[index:]
    offset = logOffset
else:
    offset = 0

for eve in main_event:
    if os.path.exists(f'{id}/{eve}.json'):
        print(f"Skipping {event} path already exist")
        continue
    file_path = f'{id}/{eve}.json'
    with open(file_path,'w') as fp:
        json.dump({}, fp)
    url = f"https://tria.ge/{id}/behavioral1/analog?main_event={eve}&offset={offset}"
    print(url)
    driver.get(url)
    time.sleep(10)

    try:
        pagination = driver.find_element(By.CLASS_NAME, "pagination")
        totalClicks = pagination.text.split(" ")[-2]
        totalClicks = int(totalClicks)
    except:
        totalClicks = 1

    for i in range(totalClicks):
        with open(file_path,'r') as fp:
            data = json.load(fp)
        data = defaultdict(list, data)
        # find elements by their class name
        elements = driver.find_element(By.CLASS_NAME, "list").find_elements(By.TAG_NAME, "li")

        for ele in elements:
            event = ele.find_element(By.TAG_NAME, "b").text
            # print(event)
            detail = ele.find_elements_by_css_selector(".key-value.rows > div")
            meta = {}
            for i in detail:
                content = i.find_elements(By.TAG_NAME, "div")
                meta[content[0].text] = content[1].text
            data[event].append(meta)

        # try to click the "Next" button and go to the next page
        try:
            pagination = driver.find_element(By.CLASS_NAME, "pagination")
            nextButton = pagination.find_elements(By.TAG_NAME, "a") 
            match = re.search(pattern, nextButton[-2].get_attribute("href"))
            if match:
                found = match.group()
                offset = int(found.split("=")[-1])
                with open(log_path, "a") as f:
                    f.write(f"\n{event} , {offset}")
                with open(file_path, 'w') as fp:
                    json.dump(data, fp)
                nextButton[-2].click()
                print("Navigating to Next Page ", found)
            else:
                print("Pattern not found in the URL.")
                break
        except Exception as e:
            print(e)
            print("Reached the end of pages.")
            break
    print(f"Finished scraping {eve}")
    
    with open(file_path, 'w') as fp:
        json.dump(data, fp)
    

Skipping Registry path already exist
Skipping Registry path already exist
https://tria.ge/231015-b71n6see63/behavioral1/analog?main_event=Files&offset=17


WebDriverException: Message: disconnected: not connected to DevTools
  (failed to check if window was closed: disconnected: not connected to DevTools)
  (Session info: chrome=118.0.5993.89)


In [79]:
print(data)

defaultdict(<class 'list'>, {'File Read': [{'proc': '23', 'path': 'C:\\Windows', 'op': 'Unknown', 'status': '0x00000000'}, {'proc': '23', 'path': 'C:\\Windows', 'op': 'OpenRead', 'status': '0x00000000'}, {'proc': '23', 'path': 'C:\\Users\\Admin\\AppData\\Local\\Temp\\', 'op': 'Unknown', 'status': '0x00000000'}, {'proc': '23', 'path': 'C:\\Windows\\Microsoft.NET\\Framework\\v4.0.30319\\', 'op': 'Unknown', 'status': '0x00000000'}, {'proc': '23', 'path': 'C:\\Windows\\Microsoft.NET\\Framework\\v4.0.30319\\', 'op': 'Unknown', 'status': '0x00000000'}, {'proc': '23', 'path': 'C:\\Windows\\Microsoft.NET\\Framework\\', 'op': 'Unknown', 'status': '0x00000000'}, {'proc': '23', 'path': 'C:\\Windows\\Microsoft.NET\\Framework\\v1.0.3705\\clr.dll', 'op': 'OpenRead', 'status': '0xc0000034'}, {'proc': '23', 'path': 'C:\\Windows\\Microsoft.NET\\Framework\\v1.0.3705\\mscorwks.dll', 'op': 'OpenRead', 'status': '0xc0000034'}, {'proc': '23', 'path': 'C:\\Windows\\Microsoft.NET\\Framework\\v1.1.4322\\clr.dl

In [89]:
# dump data to json file
import json
with open(f'{id}/Registry.json', 'w') as fp:
    json.dump(data, fp)
    

In [141]:
with open (f'temp.json','r') as fp:
    existing = json.load(fp)
existing = defaultdict(list, existing)

existing["id"].append("3")

In [142]:
existing

defaultdict(list, {'id': ['1', '3']})